In [1]:
#using VGG16 to train cats and dogs images 
from tensorflow.keras.applications import VGG16 

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))



In [2]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [9]:

# FAST FEATURE EXTRACTION WITHOUT DATA AUGMENTATION
'''
recording the output of conv_base on your data and using these outputs as
inputs to a new model.
'''
import os 
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/home/shahir/Projects/DeepLearning/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode='binary'
    )
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i* batch_size : (i+1)*batch_size] = features_batch
        labels[i*batch_size : (i+1)*batch_size] = labels_batch 
        i += 1
        if i*batch_size >= sample_count:
            break
    return features, labels 

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000) 
#You’ll feed them
#to a densely connected classifier, so first you must flatten them to (samples, 8192) :
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
#Defining and training the densely connected classifier

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
loss='binary_crossentropy',
metrics=['accuracy'])
history = model.fit(train_features, train_labels,
epochs=30,
batch_size=20,
validation_data=(validation_features, validation_labels))

Epoch 1/30
100/100 [==============================] - 2s 15ms/step - loss: 0.6872 - accuracy: 0.6019 - val_loss: 0.4414 - val_accuracy: 0.8380
Epoch 2/30
100/100 [==============================] - 1s 14ms/step - loss: 0.4385 - accuracy: 0.7916 - val_loss: 0.3646 - val_accuracy: 0.8600
Epoch 3/30
100/100 [==============================] - 1s 14ms/step - loss: 0.3510 - accuracy: 0.8549 - val_loss: 0.3310 - val_accuracy: 0.8600
Epoch 4/30
100/100 [==============================] - 1s 13ms/step - loss: 0.3317 - accuracy: 0.8524 - val_loss: 0.3000 - val_accuracy: 0.8840
Epoch 5/30
100/100 [==============================] - 1s 14ms/step - loss: 0.2762 - accuracy: 0.8871 - val_loss: 0.2864 - val_accuracy: 0.8910
Epoch 6/30
100/100 [==============================] - 1s 14ms/step - loss: 0.2542 - accuracy: 0.8986 - val_loss: 0.2749 - val_accuracy: 0.8930
Epoch 7/30
100/100 [==============================] - 1s 13ms/step - loss: 0.2547 - accuracy: 0.8965 - val_loss: 0.2688 - val_accuracy: 0.8950

In [ ]:
#Plotting the results
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()